In [61]:
import numpy as np
from scipy.stats import norm

# Create data

In [2]:
spot_init=100

int_rate = 2/100
volatility=15/100
maturity=1
nb_step=12
time_step = maturity/nb_step
nb_path=20
nb_seed=20191008

In [3]:
drift_mod = (int_rate-0.5*volatility**2)*time_step
mat_drift = drift_mod*np.cumsum(np.ones((nb_path,nb_step)),baxis=1)

In [9]:
volat_mod = volatility*np.sqrt(time_step)


In [27]:
np.random.seed(nb_seed)

In [28]:
mat_shock = np.random.standard_normal((nb_path,nb_step))

In [30]:
mat_shock_cum = np.cumsum(mat_shock,axis=1)

In [31]:
mat_volatility = volat_mod*mat_shock_cum

In [41]:
mat_volatility.shape

(20, 12)

In [33]:
mat_log_spot= mat_drift + mat_volatility


In [34]:
mat_spot_init = spot_init*np.ones((nb_path,nb_step))

In [35]:
mat_spot_fut = mat_spot_init*np.exp(mat_log_spot)

In [36]:
vec_spot_init = spot_init*np.ones((nb_path,1))

In [37]:
mat_path_spot_fut = np.concatenate((vec_spot_init,np.round(mat_spot_fut,3)),axis=1)

In [39]:
mat_path_spot_fut.shape

(20, 13)

In [49]:
mat_path_spot_fut

array([[100.   ,  96.217,  94.804,  92.972,  94.336, 100.207,  98.783,
        105.671, 114.904, 117.715, 108.745, 106.763, 101.06 ],
       [100.   , 100.255,  98.843,  96.419,  87.737,  88.071,  80.193,
         79.617,  72.863,  70.423,  71.093,  68.658,  72.335],
       [100.   ,  92.977,  93.2  ,  98.233,  96.448,  90.172,  96.604,
         99.474, 103.68 , 108.957, 115.239, 121.766, 120.964],
       [100.   , 103.328, 104.628, 103.017, 104.118, 104.356,  98.255,
         99.214,  97.686,  90.776,  87.869,  92.36 ,  98.698],
       [100.   ,  97.092,  87.424,  85.111,  84.29 ,  85.676,  79.933,
         85.14 ,  82.265,  85.025,  85.433,  84.65 ,  89.611],
       [100.   , 101.153, 100.056,  94.879,  94.155,  86.617,  88.317,
         87.081,  86.166,  85.632,  86.074,  88.473,  95.124],
       [100.   ,  98.82 ,  98.829,  95.025,  96.193,  95.001, 104.095,
        105.52 , 114.331, 111.126, 107.262, 111.02 , 116.898],
       [100.   , 102.279, 101.214, 103.897, 105.819, 102.648, 

# Question 1

In [82]:
def call_price_t(spot, strike, int_rate,time_to_maturity):
    exp_rate = np.exp(-int_rate * time_to_maturity)
    vol_mod = volatility*np.sqrt(time_to_maturity)
    d1 = (np.log(spot/strike) + (int_rate + 0.5 * volatility**2) * time_to_maturity)
    d1 = d1 / vol_mod
    d2 = d1 - vol_mod
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    price = spot * Nd1 - strike * exp_rate * Nd2
    return price
    

In [83]:
mat_call_price = np.zeros(((nb_path,nb_step + 1)))
mat_call_price.shape


(20, 13)

In [84]:
strike = spot_init

for j in range(mat_call_price.shape[0]):
    for t in range(mat_call_price.shape[1]):
        spot = mat_path_spot_fut[j,t]
        time_to_maturity = nb_step - t
        mat_call_price[j,t] = call_price_t(spot, strike, int_rate,time_to_maturity)
        mat_call_price[j,t] = round(mat_call_price[j,t],2)


<ipython-input-82-8b6b53d0ab09>:5: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = d1 / vol_mod


In [85]:
mat_call_price

array([[30.85, 26.43, 23.8 , 20.87, 20.07, 22.27, 19.28, 22.01, 26.76,
        26.49, 16.41, 11.46,  1.06],
       [30.85, 29.43, 26.71, 23.26, 15.82, 14.34,  8.51,  6.7 ,  3.  ,
         1.39,  0.61,  0.04,  0.  ],
       [30.85, 24.1 , 22.67, 24.55, 21.51, 15.61, 17.82, 17.64, 18.23,
        19.49, 21.57, 24.29, 20.96],
       [30.85, 31.79, 31.09, 28.08, 27.06, 25.29, 18.93, 17.47, 14.21,
         7.95,  4.34,  3.3 ,  0.  ],
       [30.85, 27.07, 18.79, 15.82, 13.78, 12.95,  8.39,  9.21,  6.14,
         5.39,  3.48,  1.19,  0.  ],
       [30.85, 30.11, 27.61, 22.18, 19.95, 13.49, 12.71, 10.2 ,  7.84,
         5.63,  3.7 ,  2.05,  0.  ],
       [30.85, 28.35, 26.7 , 22.28, 21.33, 18.69, 23.05, 21.9 , 26.3 ,
        21.15, 15.31, 14.78, 16.9 ],
       [30.85, 30.98, 28.48, 28.75, 28.35, 24.03, 21.  , 14.05, 13.53,
        13.59,  9.28,  3.99,  0.34],
       [30.85, 24.7 , 25.17, 22.88, 19.4 , 16.75, 21.83, 19.12, 12.91,
         7.43,  5.91,  4.32,  0.  ],
       [30.85, 27.8 , 32.25,

# Question 2

In [101]:
pnl = np.zeros(nb_path)
for j in range(len(pnl)):
    pnl[j] = mat_call_price[j,0] - np.max(mat_path_spot_fut[j,-1] - mat_spot_init[j,0],0)
pnl

array([29.79 , 58.515,  9.886, 32.152, 41.239, 35.726, 13.952, 30.511,
       40.39 , 31.518, 52.396, 44.257, 43.552, 33.602, 52.007, 11.186,
       17.02 , 28.627, 25.191,  4.69 ])

In [100]:
pnl = np.zeros(nb_path)
for j in range(len(pnl)):
    pnl[j] = mat_call_price[j,0] - mat_call_price[j,-1]
pnl

array([29.79, 30.85,  9.89, 30.85, 30.85, 30.85, 13.95, 30.51, 30.85,
       30.85, 30.85, 30.85, 30.85, 30.85, 30.85, 11.19, 17.02, 28.63,
       25.19,  4.69])

# Question 3

In [107]:
def delta( time_to_maturity, spot ):
    vol_mod = volatility*np.sqrt(time_to_maturity)
    d1 = (np.log(spot/strike) + (int_rate + 0.5 * volatility**2) * time_to_maturity)
    d1 = d1 / vol_mod
    Nd1 = norm.cdf(d1)
    return Nd1

In [105]:
mat_delta = np.zeros((nb_path,nb_step))
mat_delta.shape

(20, 12)

In [110]:
strike = spot_init

for j in range(mat_delta.shape[0]):
    for t in range(mat_delta.shape[1]):
        spot = mat_path_spot_fut[j,t]
        time_to_maturity = nb_step - t
        mat_delta[j,t] = delta(time_to_maturity, spot)
        mat_delta[j,t] = round(mat_delta[j,t],3)
mat_delta

array([[0.765, 0.73 , 0.708, 0.678, 0.674, 0.711, 0.683, 0.736, 0.811,
        0.839, 0.755, 0.74 ],
       [0.765, 0.757, 0.737, 0.707, 0.611, 0.591, 0.464, 0.415, 0.262,
        0.161, 0.094, 0.011],
       [0.765, 0.707, 0.695, 0.721, 0.693, 0.614, 0.661, 0.674, 0.704,
        0.755, 0.832, 0.936],
       [0.765, 0.775, 0.775, 0.755, 0.753, 0.745, 0.678, 0.671, 0.633,
        0.495, 0.376, 0.374],
       [0.765, 0.736, 0.646, 0.605, 0.574, 0.564, 0.46 , 0.495, 0.407,
        0.396, 0.327, 0.183],
       [0.765, 0.762, 0.745, 0.694, 0.673, 0.575, 0.568, 0.521, 0.468,
        0.407, 0.34 , 0.272],
       [0.765, 0.748, 0.737, 0.696, 0.691, 0.663, 0.732, 0.734, 0.806,
        0.778, 0.734, 0.817],
       [0.765, 0.769, 0.753, 0.761, 0.765, 0.731, 0.707, 0.609, 0.618,
        0.649, 0.582, 0.421],
       [0.765, 0.713, 0.722, 0.703, 0.665, 0.634, 0.718, 0.696, 0.605,
        0.477, 0.454, 0.443],
       [0.765, 0.743, 0.783, 0.795, 0.763, 0.81 , 0.84 , 0.821, 0.76 ,
        0.777, 0.744

# Question 4

In [111]:
mat_delta.shape

(20, 12)

In [118]:
N = 100


mat_titre = np.zeros((nb_path,nb_step))
mat_titre[:,0] = N0

for t in range(1, nb_step):
    mat_titre[:,t] = mat_titre[:,t-1] * mat_delta[:,t]
    mat_titre[:,t] = np.round(mat_titre[:,t],3)

mat_titre

array([[1.0000e+02, 7.3000e+01, 5.1684e+01, 3.5042e+01, 2.3618e+01,
        1.6792e+01, 1.1469e+01, 8.4410e+00, 6.8460e+00, 5.7440e+00,
        4.3370e+00, 3.2090e+00],
       [1.0000e+02, 7.5700e+01, 5.5791e+01, 3.9444e+01, 2.4100e+01,
        1.4243e+01, 6.6090e+00, 2.7430e+00, 7.1900e-01, 1.1600e-01,
        1.1000e-02, 0.0000e+00],
       [1.0000e+02, 7.0700e+01, 4.9136e+01, 3.5427e+01, 2.4551e+01,
        1.5074e+01, 9.9640e+00, 6.7160e+00, 4.7280e+00, 3.5700e+00,
        2.9700e+00, 2.7800e+00],
       [1.0000e+02, 7.7500e+01, 6.0062e+01, 4.5347e+01, 3.4146e+01,
        2.5439e+01, 1.7248e+01, 1.1573e+01, 7.3260e+00, 3.6260e+00,
        1.3630e+00, 5.1000e-01],
       [1.0000e+02, 7.3600e+01, 4.7546e+01, 2.8765e+01, 1.6511e+01,
        9.3120e+00, 4.2840e+00, 2.1210e+00, 8.6300e-01, 3.4200e-01,
        1.1200e-01, 2.0000e-02],
       [1.0000e+02, 7.6200e+01, 5.6769e+01, 3.9398e+01, 2.6515e+01,
        1.5246e+01, 8.6600e+00, 4.5120e+00, 2.1120e+00, 8.6000e-01,
        2.9200e-01,

# Question 5

In [119]:
pnl_covered = np.zeros(nb_path)